# GEE loading Sentinel-1 GRD

In [1]:
# NOTE: need to install ee, then earthengine-api or 
# else ee will cause conflicts
# !pip install ee
# !pip install earthengine-api
import ee
ee.Initialize()
import pandas as pd
import shapefile
import geemap
import numpy as np
import os, sys
from os.path import join

# Load event images
S1 GRD collection ID: `ee.ImageCollection("COPERNICUS/S1_GRD")` <br>
[Sentinel-1 tutorial](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-1 )

In [2]:
# load images of event of interest
region = "creoleLA"
year = "2020"
event = f"{region}_{year}"

start = '2020-01-01'
end = '2020-12-30'
eventdate = '2020-08-29'

roi = shapefile.Reader(join("..", "data", event, f"{region}.shp"))
roi.shapeType # 5: POLYGON
roi = roi.shape(0).__geo_interface__  # convert to GeoJSON
roigeom = ee.Geometry(roi)

s1 = ee.ImageCollection("COPERNICUS/S1_GRD")
avg = s1.filterBounds(roi).filterDate(start, end).first().clip(roi)
before = s1.filterBounds(roi).filterDate(start, eventdate).mean().clip(roi)
after = s1.filterBounds(roi).filterDate(eventdate, end).mean().clip(roi)

# check we have an image
avg.bandNames().getInfo()

['VV', 'VH', 'angle']

In [3]:
Map = geemap.Map()
Map.addLayer(roigeom, {}, 'ROI')
Map.centerObject(roigeom)

# Make an RGB color composite image (VV,VH,VV/VH).
rgb = ee.Image.rgb(avg.select('VV'),
                   avg.select('VH'),
                   avg.select('VV').divide(avg.select('VH')))

# Make an RGB color composite image (VV,VH,VV/VH).
rgb_before = ee.Image.rgb(before.select('VV'),
                   before.select('VH'),
                   before.select('VV').divide(before.select('VH')))

# Make an RGB color composite image (VV,VH,VV/VH).
rgb_after = ee.Image.rgb(after.select('VV'),
                   after.select('VH'),
                   after.select('VV').divide(after.select('VH')))

Map.addLayer(rgb, {'min': [-20, -20, 0], 'max': [0, 0, 2]}, 'S2 RGB')
Map.addLayer(rgb_before, {'min': [-20, -20, 0], 'max': [0, 0, 2]}, 'S2 RGB Before')
Map.addLayer(rgb_after, {'min': [-20, -20, 0], 'max': [0, 0, 2]}, 'S2 RGB After')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…